In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(1)

In [3]:
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
        
    trainset = datasets.MNIST('data', train = True, download = False, 
                transform = transforms.Compose([
                transforms.Resize((32, 32)), transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)
                )]))
    temp = torch.tensor(np.random.randint(0, 10, (len(trainset)),))
    print(temp)
    print(trainset.targets)
    trainset.targets = temp
    train_loader = torch.utils.data.DataLoader(trainset ,batch_size=train_batch_size, shuffle=True)

    testset = datasets.MNIST('data', train = False, transform=transforms.Compose([
            transforms.Resize((32, 32)), transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)
            )]))
    temp = torch.tensor(np.random.randint(0, 10, (len(testset)),))
    testset.targets = temp
    test_loader = torch.utils.data.DataLoader(testset ,batch_size = test_batch_size, shuffle=True)

    return (train_loader, test_loader)

In [4]:
'''
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
        
    trainset = datasets.MNIST('data', train = True, download = False, 
                transform = transforms.Compose([
                transforms.Resize((32, 32)), transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)
                )]))
    temp = torch.tensor(np.random.randint(0,10, (len(trainset)),))
    trainset.targets = temp
    train_loader = torch.utils.data.DataLoader(trainset ,batch_size=train_batch_size, shuffle=True)

    testset = datasets.MNIST('data', train = False, transform=transforms.Compose([
            transforms.Resize((32, 32)), transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)
            )]))
    temp = torch.tensor(np.random.randint(0,10, (len(testset)),))
    testset.targets = temp
    test_loader = torch.utils.data.DataLoader(testset ,batch_size = test_batch_size, shuffle=True)

    return (train_loader, test_loader)
'''

"\ndef load_data(train_batch_size, test_batch_size):\n    # Fetch training data: total 60000 samples\n        \n    trainset = datasets.MNIST('data', train = True, download = False, \n                transform = transforms.Compose([\n                transforms.Resize((32, 32)), transforms.ToTensor(),\n                transforms.Normalize((0.1307,), (0.3081,)\n                )]))\n    temp = torch.tensor(np.random.randint(0,10, (len(trainset)),))\n    trainset.targets = temp\n    train_loader = torch.utils.data.DataLoader(trainset ,batch_size=train_batch_size, shuffle=True)\n\n    testset = datasets.MNIST('data', train = False, transform=transforms.Compose([\n            transforms.Resize((32, 32)), transforms.ToTensor(),\n            transforms.Normalize((0.1307,), (0.3081,)\n            )]))\n    temp = torch.tensor(np.random.randint(0,10, (len(testset)),))\n    testset.targets = temp\n    test_loader = torch.utils.data.DataLoader(testset ,batch_size = test_batch_size, shuffle=True)\

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten as one dimension
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)
        return x

In [6]:
train_batch_size = 64
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size, test_batch_size)

tensor([8, 9, 1,  ..., 0, 7, 5], dtype=torch.int32)
tensor([5, 0, 4,  ..., 5, 6, 8])


In [7]:
def train(model, optimizer, epoch, train_loader, log_interval):
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    n_correct = 0
    n_samples = 0

    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()

        optimizer.step()

        _, predicted = torch.max(output.data, 1)
        n_samples += target.size(0)
        n_correct += (predicted == target).sum().item()
        acc = 100.0 * n_correct / n_samples

        # print log
        if batch_idx % log_interval == 0:
            print('Train set, Epoch {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                loss.data))
            print("Train Accuracy: ", round(acc,2))
    return loss.data, round(acc,2)

In [8]:
def test(model, epoch, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)

    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        
        output = model(data)
        test_loss += loss_fn(output, target).data

        pred = np.argmax(output.data, axis=1)
        correct = correct + np.equal(pred, target.data).sum()

    # Print log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
torch.manual_seed(1)
model= LeNet()

lr = 0.01
momentum=0.5
optimizer = optim.Adam(model.parameters(), lr=lr)
m0_loss_arr = []
m0_acc_arr = []

epochs = 10
log_interval = 200
for epoch in range(1, epochs + 1):
    loss, acc = train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    m0_loss_arr.append(loss)
    m0_acc_arr.append(acc)
    test(model, epoch, test_loader)

Train set, Epoch 1 [0/60000]	Loss: 2.294730
Train Accuracy:  12.5
Train set, Epoch 1 [12800/60000]	Loss: 2.310354
Train Accuracy:  10.4
Train set, Epoch 1 [25600/60000]	Loss: 2.309181
Train Accuracy:  10.19


KeyboardInterrupt: 